In [266]:
import math
import random
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import normalize

In [278]:
x = []
x_intercept = []
y = []

with open("wine.data", "r") as filestream:
    for line in filestream:
        currentline = line.rstrip().split(",")
        if int(currentline[0]) == 1 or int(currentline[0]) == 2:
            if int(currentline[0]) == 1:
                y.append(1)
            else:
                y.append(0)
            temp = []
            temp2 = []
            for i in range(1, len(currentline)):
                temp.append(float(currentline[i]))
                temp2.append(float(currentline[i]))
            x.append(temp)
            temp2.append(1)
            x_intercept.append(temp2)

In [279]:
# normalize the data
x_norm = normalize(x)
x_intercept_norm = normalize(x_intercept)

In [280]:
def sigmoid(z):
    return 1.0/(1 + math.exp(-z))

def my_loss_func(mytheta, myx, myy):
    returnval = 0
    
    for i in range(len(myx)):
        weighted_sum = 0
        for j in range(len(mytheta)):
            weighted_sum += mytheta[j]*myx[i][j]
        
        p = sigmoid(weighted_sum)
        
        if myy[i]:
            returnval += math.log(p)
        else:
            try:
                returnval += math.log(1 - p)
            except:
                print('p = ' + str(p))
                print('1 - p = ' + str(1 - p))
                print('log(1 - p) = ' + math.log(1 - p))
    
    return -returnval

In [281]:
def sigmoid2(scores):
    return 1 / (1 + np.exp(-scores))

def neg_log_likelihood(features, target, weights):
    scores = np.dot(features, weights)
    ll = np.sum( target*scores - np.log(1 + np.exp(scores)) )
    return -ll

In [339]:
temp = 0
mymodel = LogisticRegression(fit_intercept = False, C = 1e20)
for i in range(10000):
    mymodel.fit(x_norm, y)
    temp += neg_log_likelihood(x_norm,y,mymodel.coef_[0])
    #print (neg_log_likelihood(x_norm,y,mymodel.coef_[0]))
    
print (temp/10000)

0.613840674828


In [333]:
print (neg_log_likelihood(x_intercept_norm,y,[0]*14))
print (mymodel.coef_[0])

90.1091334728
[ -4359.18794465   3988.00661347  11575.54402613  -1737.78793499
    -52.63411038  -2794.47510264   6268.44362295  -1165.96017023
  -1598.21727083   2414.4679878   -8753.03829879    442.75865283
     67.45919174]


In [302]:
def coordinate_descent(myx, myy, threshold, israndom = False, add_intercept = False):
    if add_intercept:
        intercept = np.ones((myx.shape[0], 1))
        features = np.hstack((intercept, myx))
        
    weights = np.zeros(myx.shape[1])
    momentum = 0.9
    momentum_tracker = np.zeros(myx.shape[1])
    
    timeline = []
    
    timestep = 0
    
    factor = 0.01
    
    print ("Timestep\t\tLoss")
    
    old_log_likelihood = 1e10
    
    while (True):
        scores = np.dot(myx, weights)
        predictions = sigmoid2(scores)
        
        # Print time step and log-likelihood once in a while
        if timestep % 100000 == 0:
            if abs(neg_log_likelihood(myx, myy, weights) - old_log_likelihood) <= threshold:
                timeline.append([timestep,neg_log_likelihood(myx, myy, weights)])
                print (str(timestep)+"\t\t"+str(neg_log_likelihood(myx, myy, weights)))
                break
            if neg_log_likelihood(myx, myy, weights) - old_log_likelihood > 0:
                factor = factor*0.5
            else:
                factor = factor*1.1
            
            # Update old log likelihood
            old_log_likelihood = neg_log_likelihood(myx, myy, weights)
            timeline.append([timestep,old_log_likelihood])
            if timestep % 1000000 == 0:
                print (str(timestep)+"\t\t"+str(old_log_likelihood))
        
        # Calculate gradients
        output_error_signal = myy - predictions
        gradient = np.dot(myx.T, output_error_signal)
        
        index = None
        tempgradient = np.absolute(gradient)
        # If using random, just choose a random index for update
        if israndom:
            index = random.randint(0,len(gradient)-1)
        else:
            # Find index for largest gradient
            index = np.argmax(tempgradient)
        
        # Update weight
        #old = neg_log_likelihood(myx, myy, weights)
        oldw = weights[index]
        
        temp = factor*gradient[index] + momentum*momentum_tracker[index]
        momentum_tracker[index] = temp
        
        weights[index] = oldw + temp

        #while neg_log_likelihood(myx, myy, weights) < old:
        #    factor = factor*10
        #    weights[index] = oldw + factor*gradient[index]
            
        #weights[index] = oldw + factor*0.1*gradient[index]
        
        # Update timestep
        timestep += 1
        
    return timeline

In [303]:
mytimeline1 = coordinate_descent(np.asarray(x_norm),np.asarray(y),0.001,add_intercept=False)

Timestep		Loss
0		90.1091334728
1000000		13.7194990457
2000000		10.9024750801
3000000		8.82785625556
4000000		6.73870485229
5000000		5.33709315612
6000000		4.64169701689
7000000		4.16767510855
8000000		3.78763149503


/Users/tliu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


9000000		1472.30484149
10000000		3.45117283061
11000000		3.25475841586
12000000		3.02146120011
13000000		2.80690574109
14000000		2.61942858991
15000000		1662.64169525
16000000		2.32941199755
17000000		2.19209772217
18000000		2.06676559159
19000000		1.98842070097
20000000		1.8747265046
21000000		1698.53795429
22000000		1.69135465964
23000000		1.60111195119
24000000		1.5428286301
25000000		1.46993870734
26000000		1.44489782565
27000000		1.40152925163
28000000		1.33064173174
29000000		1380.70109852
30000000		1.23374143364
31000000		1.18194289801
32000000		1894.32843779
33000000		1.15723280004
34000000		1.11709081317
35000000		1047.66743548
36000000		1.06074092865
37000000		1.00806483381
38000000		2005.81391673
39000000		1430.54718317
40000000		0.906456369129
41000000		0.878539619545
42000000		0.840899159533
43000000		0.803779378796
44000000		0.786802929486
45000000		499.26676812
46000000		0.726113550232
47000000		0.693584107187
48000000		0.673741203103
49000000		0.641961131422
50000000		4

/Users/tliu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: overflow encountered in exp
  


54000000		0.535797121795
55000000		0.511403819932
55700000		0.510072828906


In [304]:
mytimeline2 = coordinate_descent(np.asarray(x_norm),np.asarray(y),0.001,israndom=True,add_intercept=False)

Timestep		Loss
0		90.1091334728
1000000		16.3831032024
2000000		13.4011933597
3000000		10.7108328947
4000000		8.3431907361
5000000		6.37240786305
6000000		5.37290801283
7000000		4.74301952709
8000000		4.30849148109
9000000		4.24294545095
10000000		3.83127291766
11000000		3.57930542708
12000000		3.39721043787
13000000		3.1858886511
14000000		3.0329587499
15000000		3.20086494816
16000000		2.79620290871
17000000		2.61289301184
18000000		2.51187469148
19000000		2.40075526188
20000000		2.43601389373
21000000		2.23230117229
22000000		2.15304216201
23000000		2.05454804791
24000000		1.97954608594
25000000		1.89202947782
26000000		1.8648592975
27000000		1.76486724832
28000000		1.70218695826


/Users/tliu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


29000000		1.6396130831
30000000		1.5463027154
31000000		5.02206123872
32000000		1.43106792739
33000000		1.36975783104
34000000		1.34394807973
35000000		1.26414227138
36000000		1.18339992944
37000000		7.78410372705
38000000		1.33735116552
39000000		1.1591018581
40000000		1.10215847056
41000000		0.958519869106
42000000		0.92421939806
43000000		9.93833880604
44000000		1.04812229487
45000000		0.939400476711
46000000		0.828566321684
47000000		0.859703358291
48000000		0.716200792116
49000000		93.7977460416
50000000		38.7491803925
51000000		1.02982477219
52000000		0.73871800451
53000000		0.584645219366
54000000		0.965524920728
55000000		0.455496521784
56000000		106.340321767
57000000		1.5188689708
58000000		0.397527417637
59000000		86.5959545742
60000000		0.643219691915
61000000		0.322441966894
62000000		0.798761325262
63000000		0.248895287685
64000000		1.37249209281
65000000		0.39227551189
66000000		25.3055899194
67000000		0.594120597801
68000000		0.199646462038
69000000		4.81960112268
70000

/Users/tliu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: overflow encountered in exp
  


79000000		38.0104792101
80000000		3.82343703269
81000000		1.08548204155
82000000		0.353817383102
83000000		0.161004586461


KeyboardInterrupt: 